In [1]:
# import packages

import psycopg2
import getpass
import numpy as np
import datetime
import time
import pandas as pd
import dateutil.relativedelta as dr

# connect with database

conn=psycopg2.connect(user='postgres', 
                      database='postgres', 
                      host='localhost',
                      port='5432',
                      password=getpass.getpass('Insert password'))

# check parameters

print(conn.get_dsn_parameters())

Insert password········
{'user': 'postgres', 'dbname': 'postgres', 'host': 'localhost', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'}


In [2]:
# extract data from bond_description and quotes

query = """SELECT *
            FROM (SELECT MAX("CPN") AS "CPN", "ISIN", max("BUYBACKDATE") AS "BUYBACKDATE",
                  max("BUYBACKPRICE") AS "BUYBACKPRICE"
                  FROM bonds.quotes  
                  GROUP BY "ISIN") AS a
            INNER JOIN(SELECT "ISINCode", "FaceValue", "HaveOffer", "AmortisedMty", 
                       "CouponPerYear", "NumCoupons", "EndMtyDate", "CouponType", "IssuerName"
                       FROM bonds.bond_description 
                       WHERE "Status"!='Погашен') AS b
            ON a."ISIN" = b."ISINCode";"""

# read query in python

data = pd.read_sql_query(query, conn) 

# Extract more specified data 
data.drop(['ISINCode'], axis=1)
data.BUYBACKPRICE = data.BUYBACKPRICE.fillna(0)
data = data[~data.CPN.isna()].reset_index(drop = True)
data = data[data.CPN != 0]
data = data[data.CouponPerYear > 0]

# if you had timestamp format in any of extracted fields so change format to date
# in my case it was only buybackdate

data.BUYBACKDATE = data.BUYBACKDATE.dt.date
data.head()

,CPN,ISIN,BUYBACKDATE,BUYBACKPRICE,ISINCode,FaceValue,HaveOffer,AmortisedMty,CouponPerYear,NumCoupons,EndMtyDate,CouponType,IssuerName
0,97.50,RU000A0JRYJ0,NaT,0.0,RU000A0JRYJ0,1000.0,False,False,2,9,2016-11-29,Постоянный,Автодор ГК
1,24.56,RU000A0JTG34,NaT,0.0,RU000A0JTG34,1000.0,False,True,4,20,2017-12-17,Постоянный,Воронежская Обл Прав
2,20.64,RU000A0JU823,NaT,0.0,RU000A0JU823,1000.0,False,True,4,20,2018-10-23,Фиксированный,Воронежская Обл Прав
3,29.29,RU000A0JUWK6,NaT,0.0,RU000A0JUWK6,1000.0,False,True,4,20,2019-10-14,Постоянный,Воронежская Обл Прав
4,26.55,RU000A0JS1E8,NaT,0.0,RU000A0JS1E8,1000.0,False,True,4,20,2016-12-14,Постоянный,Вологодская Обл Прав


In [3]:
# function to count coupon payments

def coupon_count(coup_str):
    full_matrix = pd.DataFrame() # local var
    end_date = coup_str.BUYBACKDATE.values[0] if coup_str.BUYBACKDATE.values[0] is not None else coup_str.EndMtyDate.values[0]
    a = int(coup_str.CouponPerYear.values[0])
    b = float(coup_str.NumCoupons.values[0])
    types = True
    if coup_str.CPN is not None or coup_str.CPN != 0:
        if coup_str.AmortisedMty.astype(str).values[0] == 'False':
            types = False
            amount = coup_str.CPN.values[0]
            for j in np.linspace((b - 1) / a, 0, b):
                date = coup_str.EndMtyDate.values[0] - dr.relativedelta(days = round(365.25 * j))
                if date >= end_date and coup_str.EndMtyDate.values[0] != end_date :
                    date = end_date
                    amount = coup_str.BUYBACKPRICE.values[0]
                    full_matrix = pd.concat([full_matrix, pd.DataFrame({date.strftime("%Y-%m-%d"): [amount]})], axis = 1)
                    break
                elif date >= end_date and coup_str.EndMtyDate.values[0] == end_date:
                    date = end_date
                    amount += coup_str.FaceValue.values[0]
                    full_matrix = pd.concat([full_matrix, pd.DataFrame({date.strftime("%Y-%m-%d"): [amount]})], axis = 1)
                    break
                full_matrix = pd.concat([full_matrix, pd.DataFrame({date.strftime("%Y-%m-%d"): [amount]})], axis = 1)
                amount = coup_str.CPN.values[0]
        else:
            percent = float(coup_str.CPN.values[0]) / float(coup_str.FaceValue.values[0])
            index = (b - 1)
            face = coup_str.FaceValue.values[0]
            while index >= 0:
                amount = percent * face
                date = coup_str.EndMtyDate.values[0] - dr.relativedelta(days = round(365.25*index/a))
                if date >= end_date and coup_str.EndMtyDate.values[0] != end_date :
                    date = end_date
                    amount = coup_str.BUYBACKPRICE.values[0]
                    full_matrix = pd.concat([full_matrix, pd.DataFrame({date.strftime("%Y-%m-%d"): [amount]})], axis = 1)
                    break
                elif date >= end_date and coup_str.EndMtyDate.values[0] == end_date:
                    date = end_date
                    amount = face
                    full_matrix = pd.concat([full_matrix, pd.DataFrame({date.strftime("%Y-%m-%d"): [amount]})], axis = 1)
                    break
                full_matrix = pd.concat([full_matrix, pd.DataFrame({date.strftime("%Y-%m-%d"): [amount]})], axis = 1)
                index -= 1
                face = face * (1 - percent)
    my_index = pd.MultiIndex.from_tuples([(coup_str.ISIN.values[0], 'Amortised' if types else 'Not Amortised'
                                           , coup_str.IssuerName.values[0] )], names=['ISIN', 'Amortisation', 'Issuer'])
    full_matrix = pd.DataFrame(full_matrix.values, index = my_index, columns = full_matrix.columns)
    return full_matrix

In [4]:
# full_matrix contains coupon payments for all the emitents
# global var:

full_matrix = pd.DataFrame()

# fill in full_matrix

for i in data.iterrows():
    full_matrix = pd.concat([full_matrix, coupon_count(pd.DataFrame(i[1]).T)], sort = True)
    
# example
full_matrix.fillna(0).head(20)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  del sys.path[0]


,,,2006-08-19,2006-10-06,2007-01-05,2007-02-18,2007-04-06,2007-06-11,2007-06-15,2007-06-16,2007-07-07,2007-08-11,...,2031-03-21,2031-09-20,2032-03-21,2032-09-19,2033-03-21,2033-09-20,2034-03-21,2034-09-20,2035-03-21,2035-09-20
ISIN,Amortisation,Issuer,,,,,,,,,,,,,,,,,,,,,
RU000A0JRYJ0,Not Amortised,Автодор ГК,0.0,0.0000,0.0000,0.0,0.0000,0.0,0.0,0.00,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RU000A0JTG34,Amortised,Воронежская Обл Прав,0.0,0.0000,0.0000,0.0,0.0000,0.0,0.0,0.00,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RU000A0JU823,Amortised,Воронежская Обл Прав,0.0,0.0000,0.0000,0.0,0.0000,0.0,0.0,0.00,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RU000A0JUWK6,Amortised,Воронежская Обл Прав,0.0,0.0000,0.0000,0.0,0.0000,0.0,0.0,0.00,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RU000A0JS1E8,Amortised,Вологодская Обл Прав,0.0,0.0000,0.0000,0.0,0.0000,0.0,0.0,0.00,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RU000A0JTF68,Amortised,Вологодская Обл Прав,0.0,0.0000,0.0000,0.0,0.0000,0.0,0.0,0.00,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RU000A0JV3M2,Amortised,Волжский Адм,0.0,0.0000,0.0000,0.0,0.0000,0.0,0.0,0.00,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RU000A0JRGR0,Amortised,Волгоградская Обл Адм,0.0,0.0000,0.0000,0.0,0.0000,0.0,0.0,0.00,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RU000A0JUP89,Amortised,Волгоградская Обл Адм,0.0,0.0000,0.0000,0.0,0.0000,0.0,0.0,0.00,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# check
print(full_matrix.values[0].T != 'nan')
print(full_matrix.shape) # table: 500 rows, 2230 columns

True
(500, 2230)


C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  


In [6]:
# from the given date to the end
import datetime
from datetime import datetime, timedelta, date
def from_date_matrix(issuer_name, date):
    spec_payments = pd.DataFrame()
    for i in range(0, len(full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, how = 'all').fillna(0).columns)):
        date_size = int(full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, \
        how = 'all').fillna(0).columns[i].split('-')[0]) * 365 + int(full_matrix.xs(str(issuer_name), \
        level = 2).dropna(axis = 1, how = 'all').fillna(0).columns[i].split('-')[1]) * 365 / 12 + \
        int(full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, how = 'all').fillna(0).columns[i].split('-')[2])
        if int(date.split('-')[0]) * 365 + int(date.split('-')[1]) * 365 / 12 + int(date.split('-')[2]) < date_size:
            data = full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, how = 'all').fillna(0).iloc[:, i:i+1]
            spec_payments = pd.concat([spec_payments, data], axis = 1)
    return spec_payments

In [7]:
# example
# insert date in format yyyy-mm-dd
from_date_matrix('Банк ВТБ','2010-01-01')

,,2010-01-05,2010-04-06,2010-07-06,2010-10-06,2011-01-05,2011-04-06,2011-07-07,2011-10-06,2012-01-05,2012-04-06,...,2016-01-05,2016-01-06,2016-01-19,2016-02-16,2016-03-24,2016-03-30,2016-06-23,2016-06-28,2016-06-29,2016-07-08
ISIN,Amortisation,,,,,,,,,,,,,,,,,,,,,
RU000A0JTK20,Not Amortised,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0,20.32,0.0,0.00,0.00,0.00,0.0,0.00,0.0
RU000A0JTN01,Not Amortised,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0,0.00,19.7,0.00,0.00,0.00,0.0,0.00,0.0
RU000A0JNGU3,Not Amortised,30.2918,30.2918,30.2918,30.2918,30.2918,30.2918,30.2918,30.2918,30.2918,30.2918,...,30.2918,100.0,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0
RU000A0JUQE1,Not Amortised,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0,0.00,0.0,23.81,0.00,23.81,100.0,0.00,0.0
RU000A0JU773,Not Amortised,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0,0.00,0.0,0.00,35.28,0.00,0.0,35.28,100.0
RU000A0JV3Q3,Not Amortised,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0


In [8]:
# payments on specified date
def on_spec_date(issuer_name, date):
    on_spec_date_payments = pd.DataFrame()
    for i in range(0,len(full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, how = 'all').fillna(0).columns)):
        date_size = int(full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, \
        how = 'all').fillna(0).columns[i].split('-')[0]) * 365 + int(full_matrix.xs(str(issuer_name),\
        level = 2).dropna(axis = 1, how = 'all').fillna(0).columns[i].split('-')[1]) * 365 / 12 + \
        int(full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, how='all').fillna(0).columns[i].split('-')[2])
        if int(date.split('-')[0]) * 365 + int(date.split('-')[1]) * 365 / 12 + int(date.split('-')[2]) == date_size:
            data = full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, how = 'all').fillna(0).iloc[:, i:i+1]
            on_spec_date_payments = pd.concat([on_spec_date_payments, data], axis = 1)
    return on_spec_date_payments

In [9]:
on_spec_date('Банк ВТБ','2016-02-16')

,,2016-02-16
ISIN,Amortisation,
RU000A0JTK20,Not Amortised,0.0
RU000A0JTN01,Not Amortised,19.7
RU000A0JNGU3,Not Amortised,0.0
RU000A0JUQE1,Not Amortised,0.0
RU000A0JU773,Not Amortised,0.0
RU000A0JV3Q3,Not Amortised,0.0


In [10]:
# from the beginning date to today

import datetime
acctual_date = datetime.datetime.now().strftime("%Y-%m-%d")

def from_date_to_today(issuer_name):
    from_date_to_today_payments = pd.DataFrame()
    for i in range(0, len(full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, how = 'all').fillna(0).columns)):
        date_size = int(full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, \
        how = 'all').fillna(0).columns[i].split('-')[0]) * 365 + \
        int(full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, how = 'all').fillna(0).columns[i].split('-')[1]) * 365/12 \
        + int(full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, how = 'all').fillna(0).columns[i].split('-')[2])
        if int(datetime.datetime.now().strftime("%Y-%m-%d").split('-')[0]) * 365 + \
        int(datetime.datetime.now().strftime("%Y-%m-%d").split('-')[1]) * 365/12 + \
        int(datetime.datetime.now().strftime("%Y-%m-%d").split('-')[2]) > date_size:
            data = full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, how = 'all').fillna(0).iloc[:, i:i+1]
            from_date_to_today_payments = pd.concat([from_date_to_today_payments, data], axis = 1)
    return from_date_to_today_payments

In [11]:
from_date_to_today('Россельхозбанк')

,,2007-08-11,2008-02-10,2008-03-28,2008-08-10,2008-09-27,2008-12-04,2009-02-09,2009-03-28,2009-05-28,2009-06-05,...,2019-04-01,2019-05-01,2019-06-28,2019-07-01,2019-07-31,2019-09-28,2019-10-01,2019-10-04,2019-10-07,2019-10-30
ISIN,Amortisation,,,,,,,,,,,,,,,,,,,,,
RU000A0JP2G4,Not Amortised,46.12,46.12,0.0000,46.12,0.0000,0.00,46.12,0.0000,0.00,0.00,...,0.000,0.0,0.000,0.000,0.0,0.000,0.000,0.0,0.0,0.0
RU000A0JUPQ7,Not Amortised,0.00,0.00,0.0000,0.00,0.0000,0.00,0.00,0.0000,0.00,0.00,...,0.000,0.0,0.000,0.000,0.0,0.000,0.000,0.0,0.0,0.0
RU000A0JUQ05,Not Amortised,0.00,0.00,0.0000,0.00,0.0000,0.00,0.00,0.0000,0.00,0.00,...,0.000,0.0,0.000,0.000,0.0,0.000,0.000,0.0,0.0,0.0
RU000A0JPF47,Not Amortised,0.00,0.00,61.5808,0.00,61.5808,0.00,0.00,61.5808,0.00,0.00,...,0.000,0.0,0.000,0.000,0.0,0.000,0.000,0.0,0.0,0.0
RU000A0JPMT8,Not Amortised,0.00,0.00,0.0000,59.34,0.0000,0.00,59.34,0.0000,0.00,0.00,...,0.000,0.0,0.000,0.000,0.0,0.000,0.000,0.0,0.0,0.0
RU000A0JPTE5,Not Amortised,0.00,0.00,0.0000,0.00,0.0000,34.41,0.00,0.0000,0.00,34.41,...,0.000,0.0,0.000,0.000,0.0,0.000,0.000,0.0,0.0,0.0
RU000A0JQ1H3,Not Amortised,0.00,0.00,0.0000,0.00,0.0000,0.00,0.00,0.0000,67.32,0.00,...,0.000,0.0,0.000,0.000,0.0,0.000,0.000,0.0,0.0,0.0
RU000A0JQK64,Not Amortised,0.00,0.00,0.0000,0.00,0.0000,0.00,0.00,0.0000,0.00,0.00,...,0.000,0.0,0.000,0.000,0.0,0.000,0.000,0.0,0.0,0.0
RU000A0JQK72,Not Amortised,0.00,0.00,0.0000,0.00,0.0000,0.00,0.00,0.0000,0.00,0.00,...,0.000,0.0,0.000,0.000,0.0,0.000,0.000,0.0,0.0,0.0


In [12]:
# BONUS: from the given date to the end

def crazy(issuer_name, date, db_login, from_date_matrix):
    conn = psycopg2.connect(user = db_login['user'],
                            database = db_login['database'], 
                            host = db_login['host'], 
                            port = db_login['port'], 
                            password = getpass.getpass('Insert password: '))

    query = """SELECT *
            FROM (SELECT MAX("CPN") AS "CPN", "ISIN", max("BUYBACKDATE") AS "BUYBACKDATE",
                  max("BUYBACKPRICE") AS "BUYBACKPRICE"
                  FROM bonds.quotes  
                  GROUP BY "ISIN") AS a
            INNER JOIN(SELECT "ISINCode", "FaceValue", "HaveOffer", "AmortisedMty", 
                       "CouponPerYear", "NumCoupons", "EndMtyDate", "CouponType", "IssuerName"
                       FROM bonds.bond_description 
                       WHERE "Status"!='Погашен') AS b
            ON a."ISIN" = b."ISINCode";"""

    # read query in python

    data = pd.read_sql_query(query, conn) 

    # Extract more specified data 
    data.drop(['ISINCode'], axis=1)
    data.BUYBACKPRICE = data.BUYBACKPRICE.fillna(0)
    data = data[~data.CPN.isna()].reset_index(drop = True)
    data = data[data.CPN != 0]
    data = data[data.CouponPerYear > 0]

    # if you had timestamp format in any of extracted fields so change format to date
    # in my case it was only buybackdate

    data.BUYBACKDATE = data.BUYBACKDATE.dt.date
    
    def coupon_count(coup_str):
        full_matrix = pd.DataFrame() # local var
        end_date = coup_str.BUYBACKDATE.values[0] if coup_str.BUYBACKDATE.values[0] is not None else coup_str.EndMtyDate.values[0]
        a = int(coup_str.CouponPerYear.values[0])
        b = float(coup_str.NumCoupons.values[0])
        types = True
        if coup_str.CPN is not None or coup_str.CPN != 0:
            if coup_str.AmortisedMty.astype(str).values[0] == 'False':
                types = False
                amount = coup_str.CPN.values[0]
                for j in np.linspace((b - 1) / a, 0, b):
                    date = coup_str.EndMtyDate.values[0] - dr.relativedelta(days = round(365.25 * j))
                    if date >= end_date and coup_str.EndMtyDate.values[0] != end_date :
                        date = end_date
                        amount = coup_str.BUYBACKPRICE.values[0]
                        full_matrix = pd.concat([full_matrix, pd.DataFrame({date.strftime("%Y-%m-%d"): [amount]})], axis = 1)
                        break
                    elif date >= end_date and coup_str.EndMtyDate.values[0] == end_date:
                        date = end_date
                        amount += coup_str.FaceValue.values[0]
                        full_matrix = pd.concat([full_matrix, pd.DataFrame({date.strftime("%Y-%m-%d"): [amount]})], axis = 1)
                        break
                    full_matrix = pd.concat([full_matrix, pd.DataFrame({date.strftime("%Y-%m-%d"): [amount]})], axis = 1)
                    amount = coup_str.CPN.values[0]
            else:
                percent = float(coup_str.CPN.values[0]) / float(coup_str.FaceValue.values[0])
                index = (b - 1)
                face = coup_str.FaceValue.values[0]
                while index >= 0:
                    amount = percent * face
                    date = coup_str.EndMtyDate.values[0] - dr.relativedelta(days = round(365.25*index/a))
                    if date >= end_date and coup_str.EndMtyDate.values[0] != end_date :
                        date = end_date
                        amount = coup_str.BUYBACKPRICE.values[0]
                        full_matrix = pd.concat([full_matrix, pd.DataFrame({date.strftime("%Y-%m-%d"): [amount]})], axis = 1)
                        break
                    elif date >= end_date and coup_str.EndMtyDate.values[0] == end_date:
                        date = end_date
                        amount = face
                        full_matrix = pd.concat([full_matrix, pd.DataFrame({date.strftime("%Y-%m-%d"): [amount]})], axis = 1)
                        break
                    full_matrix = pd.concat([full_matrix, pd.DataFrame({date.strftime("%Y-%m-%d"): [amount]})], axis = 1)
                    index -= 1
                    face = face * (1 - percent)
        my_index = pd.MultiIndex.from_tuples([(coup_str.ISIN.values[0], 'Amortised' if types else 'Not Amortised'
                                               , coup_str.IssuerName.values[0] )], names=['ISIN', 'Amortisation', 'Issuer'])
        full_matrix = pd.DataFrame(full_matrix.values, index = my_index, columns = full_matrix.columns)
        return full_matrix
    
    full_matrix = pd.DataFrame()

    # fill in full_matrix

    for i in data.iterrows():
        full_matrix = pd.concat([full_matrix, coupon_count(pd.DataFrame(i[1]).T)], sort = True)
        full_matrix = full_matrix.fillna(0)
    
    import datetime
    def from_date_matrix(issuer_name, date):
        spec_payments = pd.DataFrame()
        for i in range(0, len(full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, how = 'all').fillna(0).columns)):
            date_size = int(full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, \
            how = 'all').fillna(0).columns[i].split('-')[0]) * 365 + int(full_matrix.xs(str(issuer_name), \
            level = 2).dropna(axis = 1, how = 'all').fillna(0).columns[i].split('-')[1]) * 365/12 + \
            int(full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, how = 'all').fillna(0).columns[i].split('-')[2])
            if int(date.split('-')[0]) * 365 + int(date.split('-')[1]) * 365/12 + int(date.split('-')[2]) < date_size:
                data = full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, how = 'all').fillna(0).iloc[:, i:i+1]
                spec_payments = pd.concat([spec_payments, data], axis = 1)
        return spec_payments
    
    results = {'ISSUER': issuer_name, 'DATE': date, 'MATRIX_OF_PAYMENTS': from_date_matrix(issuer_name, date)}
    return results

In [13]:
login = {'user': 'postgres', 'database': 'postgres', 'host': 'localhost', 'port': '5432'}
x = crazy('Банк ВТБ','2010-01-01', login, from_date_matrix)
print(x['ISSUER'], x['DATE'])
x['MATRIX_OF_PAYMENTS'].loc[:, (x['MATRIX_OF_PAYMENTS'] != 0).any(axis = 0)]

Insert password: ········


C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:47: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.


Банк ВТБ 2010-01-01


,,2010-01-05,2010-04-06,2010-07-06,2010-10-06,2011-01-05,2011-04-06,2011-07-07,2011-10-06,2012-01-05,2012-04-06,...,2016-01-05,2016-01-06,2016-01-19,2016-02-16,2016-03-24,2016-03-30,2016-06-23,2016-06-28,2016-06-29,2016-07-08
ISIN,Amortisation,,,,,,,,,,,,,,,,,,,,,
RU000A0JTK20,Not Amortised,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0,20.32,0.0,0.00,0.00,0.00,0.0,0.00,0.0
RU000A0JTN01,Not Amortised,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0,0.00,19.7,0.00,0.00,0.00,0.0,0.00,0.0
RU000A0JNGU3,Not Amortised,30.2918,30.2918,30.2918,30.2918,30.2918,30.2918,30.2918,30.2918,30.2918,30.2918,...,30.2918,100.0,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0
RU000A0JUQE1,Not Amortised,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0,0.00,0.0,23.81,0.00,23.81,100.0,0.00,0.0
RU000A0JU773,Not Amortised,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0,0.00,0.0,0.00,35.28,0.00,0.0,35.28,100.0
RU000A0JV3Q3,Not Amortised,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.0
